# Generación base de datos
Este notebook trata de generar la base de datos que utilizamos en nuestro proyecto. Para ello utilizamos unos archivos json que se encuentran en el repositorio de [GitHub](https://github.com/Nieterulz/nfr-extraction) del trabajo.

### Activamos Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importamos librerías necesarias

In [2]:
import json
import pandas as pd
import os

### Definimos una función que devolverá el contenido que necesitamos del archivo JSON que le pasemos

In [3]:
def readJson(file):
    jsonFile = open(RUTA_JSON + file, "r", encoding="utf8")
    jsonData = jsonFile.read()
    jsonData = json.loads(jsonData)
    jsonData = jsonData['content']
    return jsonData

### Definimos una función que devolverá un diccionario con los valores de clasificación para cada categoría

In [4]:
def readClassifications(booleanClassifications):
    classifications = {}

    for c in categories:
        try:
            classifications[c] = int(
                booleanClassifications[c]["value"] == "TRUE")
            pass
        except Exception:
            classifications[c] = 0
            pass

    classifications["not applicable"] = 1 if(sum(classifications.values()) == 0) else 0

    return classifications

### Definimos una función que devolverá un dataframe con los datos que necesitamos del archivo json que pasemos como parámetro 

In [5]:
def generateDataframe(file):
    jsonData = readJson(file)
    file = file.replace('.json', '')
    columns = ['sentences'] + categories
    db = pd.DataFrame(columns=columns)

    for element in jsonData:
        sentence = element['sentence']
        classification = readClassifications(element['booleanClassifications'])
        row = {
            'sentences': sentence,
        }
        for c in categories:
            row[c] = classification[c]

        db = db.append(row, ignore_index=True)

    return db

### El siguiente código se encarga de generar el archivo excel que será nuestra base de datos

In [6]:
RUTA_JSON = "./drive/My Drive/nfr-extraction/json/"
RUTA_DB = "./drive/My Drive/nfr-extraction/"

files = os.listdir(RUTA_JSON)

categories = [
    "access control", "audit", "availability", "legal", "look and feel",
    "maintainability", "operational", "privacy", "recoverability", "capacity and performance",
    "reliability", "security", "usability", "other nonfunctional", "functional", "not applicable"
]

columns = ['sentences'] + categories

db = pd.DataFrame(columns=columns)

for file in files:
    print("Procesando", file, "...")
    db = db.append(generateDataframe(file), ignore_index=True)

db.to_excel(RUTA_DB + "db.xlsx")
print("\n¡Base de datos generada con éxito!")

Procesando CCHIT.json ...
Procesando CMS-DUA.json ...
Procesando EMR-RFP.json ...
Procesando HIPAA.json ...
Procesando LACountyEHR_Requirements.json ...
Procesando Meaningful.json ...
Procesando NC_DETECT_DUA.json ...
Procesando OpenEMR_4.1_Users_Guide.json ...
Procesando OpenEMR_4_1_1_Linux_Installation.json ...
Procesando eCFR.json ...
Procesando iTrust.json ...
Procesando promisedataALL.json ...
¡Base de datos generada con éxito!
